In [1]:
import math
import numpy as np
import cmath 
import skrf as rf


In [77]:
fo = 100e6
wo = 2*np.pi*fo

y11 =   5.00e-3 + 1.50e-3*1j
y12 = - 0.012e-3 - 0.02e-3*1j
y21 =   5.50e-3 - 0.38e-3*1j 
y22 =   0.04e-3 + 0.70e-3*1j

ytr = np.matrix([[y11, y12],[ y21, y22]])


In [78]:
def printY(y):
    print( 'y11= {0.real:4.2E}+j({0.imag:4.2E})' .format(y[0,0]) )
    print( 'y12= {0.real:4.2E}+j({0.imag:4.2E})' .format(y[0,1]) )
    print( 'y21= {0.real:4.2E}+j({0.imag:4.2E})' .format(y[1,0]) )
    print( 'y22= {0.real:4.2E}+j({0.imag:4.2E})' .format(y[1,1]) )

def printYp(y):
    print('Polar')
    print( '|y11|= {0:6.4E}, <y11= {1:6.1F}' .format(abs(y[0,0]), cmath.phase(ytr[0,0])*180./np.pi) )
    print( '|y12|= {0:6.4E}, <y12= {1:6.1F}' .format(abs(y[0,1]), cmath.phase(ytr[0,1])*180./np.pi) )
    print( '|y21|= {0:6.4E}, <y21= {1:6.1F}' .format(abs(y[1,0]), cmath.phase(ytr[1,0])*180./np.pi) )
    print( '|y22|= {0:6.4E}, <y22= {1:6.1F}' .format(abs(y[1,1]), cmath.phase(ytr[1,1])*180./np.pi) )

    
def getSpice(y):
    print( '.param g11= {0.real:6.4}' .format(y[0,0]) )
    print( '.param g12= {0.real:6.4E}' .format(y[0,1]) )
    print( '.param g21= {0.real:6.4E}' .format(y[1,0]) )
    print( '.param g22= {0.real:6.4E}' .format(y[1,1]) )
    print( '.param c11= {0:6.4E}' .format(y[0,0].imag/(wo)) )
    print( '.param c12= {0:6.4E}' .format((y[0,1].imag)/(wo)) )
    print( '.param c21= {0:6.4E}' .format((y[1,0].imag)/(wo)) )
    print( '.param c22= {0:6.4E}' .format(y[1,1].imag/(wo)) )
    
def printInOut(yg,yl):
    print('######## Entrada #########')
    print( 'rin= {0:4.2E}' .format(1/yg.real) )
    print( 'cin= {0:4.2E}' .format(-yg.imag/wo) )
    print('######## Salida #########')
    print( 'ro= {0:4.2E}' .format(1/yl.real) )
    print( 'co= {0:4.2E}' .format(-yl.imag/wo) )


In [87]:
def linvill(y):
    ya = y[0,1]*y[1,0]
    C = ( abs(ya) + ya.real )/ (2*y[0,0].real*y[1,1].real)
    print('###############  LINVILL   #####################')
    print( 'coeficiente de linvill C= {:4.2F}' .format(C))
    if C <= 1: print( 'si C es menor a uno es incondicionalmente estable')
    else    : print( 'si C es mayor o igula a 1 es potencialmente inestable')
    
    
def stern(y, yg, yl):
    ya = y[0,1]* y[1,0]
    K = (2*(yg.real+y[0,0].real)*(yl.real+y[1,1].real))/( abs(ya) + ya.real )
    print('###############  Stern   #####################')
    print('coeficiente de Stern K=', K)
    print('si k es MAYOR a 1 no oscilara' )   

def Ar(y,yg,yo): # vo/ig
    ya = y[0,1]* y[1,0]
    Ar = -y[1,0]/((y[0,0]+yg)*(y[1,1]+yl)-ya)
    print('###############  Ganancia de transresistencia (vo/ig)   #########')
    print( 'vo/ig = {:4.2F}' .format(abs(Ar)))
    print( 'vo/ig = {:4.2F} dB' .format(20*np.log10(abs(Ar)))) 
    
def At(y,yg,yl) :  
    ya = y[0,1]* y[1,0]
    pgain = 4*yg.real*yl.real*abs(y[1,0])**2 / (abs((yl+y[1,1])*(y[0,0]+yg)-ya))**2
    print('###############  Ganancia de potencia   #####################')
    print( 'Power Gain = {:4.2F}' .format(pgain))
    print( 'Power Gain = {:4.2F} dB' .format(10*np.log10(pgain)))    
    
    
def neutraliza(y,wo,n = 1):
    Cn = -y[0,1].imag/wo*n
    y11 =  wo*Cn*1j
    y12 =  wo*Cn*1j/n
    y21 =  wo*Cn*1j/n
    y22 =  wo*Cn*1j/n**2
    yn  = np.matrix([[y11, y12],[ y21, y22]])
    # cuadripolo resultante
    yt  = y + yn
    print('cap=',Cn, 'n=', n)
    return yt

def yg_yl(y):
    yl = y[1,1].real - y[1,1].imag*1j
    for i in range(20):
        yin  = y[0,0] - y[0,1]*y[1,0]/(y[1,1]+yl)
        yg   = yin.real - yin.imag*1j
        yout = y[1,1] - y[0,1]*y[0,1]/(y[0,0]+yg)
        yl   = yout.real - yout.imag*1j
    return (yg, yl)


def divC(Q,wo,Rin,RL, Cext = 0):
    Qm = ((RL/Rin)*(1+Q**2)-1)**0.5
    C  = Q/Rin/wo - Cext
    Cs = C*(1+1/Q**2)
    Cb = Qm/RL/wo
    Cbs = Cb*(1+1/Qm**2)
    Ca = 1/(1/Cs-1/Cbs)
    return(Ca,Cb)

In [80]:
printY(ytr)

# calculo linvill del transistor
linvill(ytr)

# admitancia de entrada y de salida para MTE
(yg,yl) = yg_yl(ytr) 

printInOut(yg,yl)

# stern 
stern(ytr, yg, yl)

Ar(ytr,yg,yl)
At(ytr,yg,yl)

y11= 5.00E-03+j(1.50E-03)
y12= -1.20E-05+j(-2.00E-05)
y21= 5.50E-03+j(-3.80E-04)
y22= 4.00E-05+j(7.00E-04)
###############  LINVILL   #####################
coeficiente de linvill C= 0.14
si C es menor a uno es incondicionalmente estable
######## Entrada #########
rin= 1.69E+02
cin= 4.48E-12
######## Salida #########
ro= 2.50E+04
co= 1.11E-12
###############  Stern   #####################
coeficiente de Stern K= 31.7871714588
si k es MAYOR a 1 no oscilara
###############  Ganancia de transresistencia (vo/ig)   #########
vo/ig = 5818.03
vo/ig = 75.30 dB
###############  Ganancia de potencia   #####################
Power Gain = 32.09
Power Gain = 15.06 dB


Dos etapas sintonizadas sincronicas, realizadas con 1 transistor.
Qc = 32 
fo = 100MHz

In [81]:
rg = 50
rl = 1500

Qo =100
Qc = 32

# dado el transistor
ri = 1/yg.real
bi = -yg.imag
ro = 1/yl.real
bo = -yl.imag






47.05882352941177
23.0621841326
L2= 4.22456096053e-07
C2a= 7.94204365587e-12
C2b= 2.44697373546e-11
(7.940661843109191e-12, 2.4448105255296896e-11)


In [88]:

## Salida
# calculo Q sintonizados
Qs = 1/(1/Qc-1/Qo)
print(Qs)

# Calculo de componentes
rext = ro/2
L2 = (1/Qs)*rext/wo 

C2 = 1/(wo**2*L2) - bo/wo

# divisor capaticitivo
# forma aproximada
n = (ro/rl)**0.5
Qm = 2*Qs/n
print(Qm)
C2a = C2 *n/(n-1)
C2b = C2 *n

# Muestra resultados
print('L2=',L2)
print('C2a=',C2a)
print('C2b=',C2b)

# sin aproximar
print(divC(Qs*2,wo,ro,rl, bo/wo))



47.05882352941177
23.0621841326
L2= 4.22456096053e-07
C2a= 6.46645058031e-12
C2b= 1.99233791922e-11
(6.0980279632137782e-12, 2.4448105255296896e-11)


In [94]:
# Q de los sintonizados
Qs  = 1/(1/Qc-1/Qo)
# R tope de bobina
rt = ro
# calculo Adaptador de entrada
rext = rt/2
L1 = (1/Qs)*rext/wo 

# transformador
n = (rt/ri)**0.5
L1a = L1/n**2 


C1 = 1/(wo**2*L1) - (bi/n**2)/wo



# divisor capaticitivo
n = (rt/rg)**0.5
Qm = 2*Qs/n
print(Qm)
C1a = C1 *n/(n-1)
C1b = C1 *n

# sin aproximar
#C1a,C1b = divC(Qs*2,wo,rt,rg)

print('L1=',L1)
print('L1a=',L1a)
print('C1a=',C1a)
print('C1b=',C1b)



4.21055949158
L1= 4.22456096053e-07
L1a= 2.85629010467e-09
C1a= 6.24503090207e-12
C1b= 1.33348677603e-10


In [95]:
# transistor

getSpice(ytr)



.param g11=  0.005
.param g12= -1.2000E-05
.param g21= 5.5000E-03
.param g22= 4.0000E-05
.param c11= 2.3873E-12
.param c12= -3.1831E-14
.param c21= -6.0479E-13
.param c22= 1.1141E-12


In [96]:
# sintonizado entrada
print( '.param c1a= {0:6.4E}' .format(C1a))
print( '.param c1b= {0:6.4E}' .format(C1b))
print( '.param L1= {0:6.4E}' .format(L1))
print( '.param L1a= {0:6.4E}' .format(L1a))
print( '.param rp1= {0:6.4E}' .format(Qo*L1*wo))


.param c1a= 6.2450E-12
.param c1b= 1.3335E-10
.param L1= 4.2246E-07
.param L1a= 2.8563E-09
.param rp1= 2.6544E+04


In [97]:
# sintonizado salida
print( '.param c2a= {0:6.4E}' .format(C2a))
print( '.param c2b= {0:6.4E}' .format(C2b))
print( '.param L2= {0:6.4E}' .format(L2))
print( '.param rp2= {0:6.4E}' .format(Qo*L2*wo))

.param c2a= 6.4665E-12
.param c2b= 1.9923E-11
.param L2= 4.2246E-07
.param rp2= 2.6544E+04
